# IMDB 영화평데이터 > 감성 분류를 위한 트랜스포커 아키텍처 모델 구축

1. 정수토큰 시퀀스(길이 80) 입력
2. 토큰 임베딩 + 위치 임베딩
3. 멀티헤드어텐션 3헤드
4. concate + 정규화
5. FFN (Dense + Dense)
6. concate + 정규화
7. 분류기 (Dense)

## 1. 정수토큰 시퀀스(길이 80) 입력

In [2]:
import tensorflow as tf
from tensorflow.keras import Model, layers

## 2. 토큰 임베딩 + 위치 임베딩

In [4]:
# 토큰 임베딩
inputs = layers.Input(shape=(80, ))
input_embedding = layers.Embedding(input_dim=1000, output_dim=32)(inputs)

In [6]:
# 위치 임베딩
positions = tf.range(start=0, limit=80)
pos_embedding = layers.Embedding(input_dim=80, output_dim=32)(positions)
pos_enc_output = pos_embedding + input_embedding

## 3. 멀티헤드어텐션 3헤드

In [9]:
# K, V를 받아주는 구조
attention_output = layers.MultiHeadAttention(num_heads=3, key_dim=32)(pos_enc_output, pos_enc_output)   # output이 그대로 들어가야하므로 32개로 통일

## 4. concate + 정규화

In [10]:
x = layers.add([pos_enc_output, attention_output])
x = layers.BatchNormalization()(x)   # 정규화

## 5. FFN (Dense + Dense)

In [12]:
from tensorflow.keras.models import Sequential
ffnn = Sequential(
    [
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu')
    ]
)(x)

## 6. concate + 정규화

In [14]:
x = layers.add([ffnn, x])
x = layers.BatchNormalization()(x)

## 7. 분류기 (Dense)

In [15]:
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0, 1)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0, 1)(x)
outputs = layers.Dense(2, activation='softmax')(x)

# 모델 구성

In [17]:
model = Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 80)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 80, 32)    │     32,000 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 80, 32)    │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 80, 32)    │     12,608 │ add_1[0][0],      │
│ (MultiHeadAttentio… │                   │            │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 80, 32)    │          0 │ add_1[0][0],      │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 80, 32)    │        128 │ add_2[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 80, 32)    │      4,192 │ batch_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 80, 32)    │          0 │ sequential[0][0], │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 80, 32)    │        128 │ add_3[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 80, 32)    │      4,192 │ batch_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 80, 32)    │          0 │ sequential_1[0][… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 80, 32)    │        128 │ add_4[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_5 (Add)         │ (None, 80, 32)    │          0 │ sequential_1[0][… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 80, 32)    │        128 │ add_5[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 32)        │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 32)        │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │      2,112 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 64)        │          0 │ dense_4[0][0]   

 Total params: 55,746 (217.76 KB)

 Trainable params: 55,490 (216.76 KB)

 Non-trainable params: 256 (1.00 KB)

In [18]:
# 손실함수와, 옵티마이저 지정
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# imdb data load

In [20]:
from tensorflow.keras.datasets import imdb
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)
(X_train, y_train), (X_test, y_test)

((array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
         list([1, 194, 1153, 194, 8255, 78, 

In [21]:
# 텍스트 데이터의 전처리
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train_pad = pad_sequences(X_train, maxlen=80, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test, maxlen=80, padding='post', truncating='post')

In [32]:
model.fit(X_train_pad, y_train, epochs=20, batch_size=30)

Epoch 1/20


2025-09-05 12:40:22.993027: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-09-05 12:40:22.993099: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-09-05 12:40:22.993169: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-09-05 12:40:22.993233: I external/l

828/834 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7854 - loss: 0.6999

2025-09-05 12:40:31.213298: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-09-05 12:40:31.213356: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-09-05 12:40:31.213402: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-09-05 12:40:31.213443: I external/l

834/834 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.7909 - loss: 0.5246
Epoch 2/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8455 - loss: 0.3447
Epoch 3/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8688 - loss: 0.3004
Epoch 4/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8781 - loss: 0.2752
Epoch 5/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8907 - loss: 0.2553
Epoch 6/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8935 - loss: 0.2446
Epoch 7/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9036 - loss: 0.2264
Epoch 8/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9104 - loss: 0.2126
Epoch 9/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9116 - loss: 0.2080
Epoch 10/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9207 - loss: 0.1896
Epoch 11/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9226 - loss: 0.1846
Epoch 12/20
834/834 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accurac

In [31]:
model.evaluate(X_test_pad, y_test)

782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7320 - loss: 2.5047


[2.5047287940979004, 0.7319599986076355]

- epochs=20, batch_size=200 : accuracy: 0.6694 - loss: 0.9977   
- epochs=30, batch_size=200 : accuracy: 0.6998 - loss: 1.8136
- epochs=40, batch_size=200 : accuracy: 0.6993 - loss: 1.9431
- epochs=50, batch_size=200 : accuracy: 0.7320 - loss: 2.5047

In [26]:
import numpy as np
pred = model.predict(X_test_pad)
pred = np.argmax(pred, axis=1)

782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step


In [27]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred)

array([[8768, 3732],
       [3773, 8727]])